<b>Imports. The "limit" parameter in keyedvectors.load_word2vec_format will give you the n most common words </b>

In [ ]:
from gensim.models.keyedvectors import KeyedVectors
word_vectors = KeyedVectors.load_word2vec_format(r"C:\Users\mikef\Downloads\GoogleNews-vectors-negative300.bin.gz", binary=True, limit=15000)

<b>Run nltk.download() to download brown and wordnet. Only needs to be done once.</b>

In [ ]:
import nltk
#nltk.download()

In [ ]:
import random
import inflect
from nltk.corpus import brown
import spacy
import re

In [ ]:
nlp = spacy.load("en_core_web_sm")

Some commented out test code

In [ ]:
#old code for getting sentences in serial that contain inputted word
# def sentence_word(word):
#     word_sents = []
#     for i in brown.sents(categories='fiction'):
#         if word in i:
#             word_sents.append(i)
#     return word_sents

In [ ]:
#word_vectors.most_similar(positive=['said'])

In [ ]:
#1 - word_vectors.distance('solicitation','envelope')

<h1>Main Functions</h1>

<p>Generates a list of "semantic" "opposites" of a word. List items are randomized so that not every instance of a word, say 'dog', has the same opposite on each occurence. </p>

In [ ]:
def opposite_vector(word):
    try:
        zz = word_vectors.most_similar(negative=[word],topn=1000)
        random.shuffle(zz)
        return zz
    except:
        return 0

<p>Replaces words in a sentence with their "semantic" "opposites". Only replaces words of selected part of speech categories. Input the sentence as a list, and input the desired part of speech categories as a list as well. POS categories must be inputed as follows: Noun = 'NOUN', Verb = 'VERB', Adjective = 'ADJ'. Example: opposite_sentence(['Look', 'at', 'the', 'funny', 'cat'], ['NOUN','ADJ']) will try to switch all the nouns and adjectives in the sentence "Look at the funny cat" with their opposites. </p>

In [ ]:
def opposite_sentence(sentence,cats=['NOUN']): #sentence inputted as list, cats = pos categories
    if isinstance(sentence,str):
        return opposite_sentence(re.split("[ ,]",sentence))
    a = nlp(' '.join(sentence.copy())) #a = list-like object of words that can now be queried for their POS (eg a[0].pos_)
    return_sent = sentence.copy()
    replacements = []
    
    ###This whole 'skip' var is needed since apostrephes are considered as parts of speech.
    ###This leads to nlp(sentence) having a longer length then sentence.split()
    skip = 0
    
    for i in range(len(a)): #for each word (or punctuation) in sentence
        pos = a[i].pos_     #get POS of word 
        if pos == 'PART':
            skip +=1
        if pos not in cats: #If POS not in the inputed categories then ignore it
            continue
        opposite_list = opposite_vector(str(a[i])) #Get the words list of opposite words
        if opposite_list == 0:                     #Unless we can't find a list of opposite words  
            continue
#Now iterate over each opposite word. Try replacing it with the original word in the sentence.
#If the opposite word takes on the same POS when replaced into the sentence, finalize it as the official replacement word
        for opposite in [o[0] for o in opposite_list]: 
            c = return_sent.copy() #get current return sentence 
            c[i-skip] = opposite        #replace current word with opposite
#This code may be useful if theres issues with singular/plural nouns -- will look into more later
#             if inflect.engine().singular_noun(opposite) != False:
#                 continue     
            if nlp(' '.join(c))[i].pos_== pos: #check if replaced opposite has same POS or original word
                replacements.append((i-skip,opposite)) #if so, officially replace it
                break
    for i in replacements: 
        return_sent[i[0]] = "[" + i[1] + "]" #i[0] is index, i[1] is string replacement
    return return_sent

<h1>Display Function(s)</h1>

In [ ]:
def print_x(total_sents,cats):
    for i in range(total_sents):
        q = random.choice(brown.sents())
        try:
            a = ' '.join(q)
            b = ' '.join(opposite_sentence(q,cats))
            if a != b:
                print('*********************************************')
                print(a)
                print()
                print(b)
                print('*********************************************')
        except:
            pass